## World Bank Data Project

placeholder project description

QA Project for World Bank Survey Dataset - World bank has compiled a survey asking specific financial questions to various federal banks across the globe. This project will train a NLP generative model on the dataset and allow the user to ask questions related to the information.  

step 1 - load excel file into python  
step 2 - process excel file into a NLP readable format - question:answer pairs  
step 3 - PHASE 1 - run pii identifying model (microsoft presidio) on dataset to clean it  
step 4 - PHASE 2 - train NLP model (Google T5 via pyTorch) on dataset  
step 5 - test and finetune model  
step 6 - upload model to azure  

Future Steps:  
- PHASE 3
    - implement API for interacting with model
    - add some sort of sentiment analysis to clasify questions/answers (financial questions, admin questions, etc)
    - get feedback on model performance (answer quality/hallucinations/knowledge gaps)  
    - add additional survey questions to knowledge base  

CICD: GitHub

In [ ]:
# download data from World Bank Database
import requests

url = "https://datacatalogfiles.worldbank.org/ddh-published/0038632/2/DR0047737/2021_04_26_brss-public-release.xlsx"
response = requests.get(url)

with open("worldbank_data.xlsx", "wb") as f:
    f.write(response.content)

convert xlsx file from row column format to something that works for t5 training  
need to use question:answer pairs  
model treats multi-select questions as independent binary questions for each option

In [ ]:
## read and process data
import pandas as pd
import re

# loads all sheets at once
allSheets = pd.ExcelFile("worldbank_data.xlsx")

# store samples
samples = []

# process all sheets except first 2 and last 1
process = allSheets.sheet_names[2:-1]

# read first sheet and extract countries
dfFirst = pd.read_excel(allSheets, sheet_name=process[0], header=None)
countries = [str(c) for c in dfFirst.iloc[0, 2:].values if not pd.isna(c)]

for sheet in process:
    # read current sheet
    df = pd.read_excel(allSheets, sheet_name=sheet, header=None)

    # create parent and base vars
    parent = None
    currBase = None

    # iterate through every row except header
    # get question index and question text
    for idx, row in df.iloc[1:].iterrows():
        qIndex = row[0]
        qText = row[1]

        # if the question index is null but text does exist 
        # then the question is a parent question
        # assign parent question and then clear prev base and move onto next row
        if pd.isna(qIndex) and not pd.isna(qText):
            parent = qText
            currBase = None
            continue

        # regex starts with Q and captures groups delimited by _
        # group 1 is the main question number
        # group 2 is sub-question number
        # group 3 is for multi-part questions with extra text
        # non-capturing group is for sections of index which are unnecessary
        match = re.match(r'Q(\d+)_([0-9_]+?)([a-zA-Z_]+)?(?:_[A-Z]|_\d{4}|$)', str(qIndex))

        # if regex matched then process row, otherwise skip
        if match:
            baseNum = f"{match.group(1)}_{match.group(2)}"
            isMulti = bool(match.group(3)) or bool(re.search(r'_\d{4}', str(qIndex)))
            part = match.group(3) if match.group(3) else ""
        else:
            continue

        # if new base is different to current base, update base
        if baseNum and baseNum != currBase:
            # reset parent if new question isn't multi part
            if not isMulti:
                parent = None
            currBase = baseNum

        # loop through each column
        for colIdx, country in enumerate(countries):

            # get answer for current column
            answer = row[colIdx + 2]
            
            # skip column if there's no answer
            if pd.isna(answer):
                continue

            # if question is multi-part combine parent question and question text
            if isMulti and parent:
                completeQ = f"{parent} {qText}"
            
            # otherwise just append question text
            else:
                completeQ = qText

            # fill in sample entry
            sample = {
                "input": f"Answer this question about {country}: {completeQ}".strip(),
                "target": str(answer).strip()
            }

            # append sample to list
            samples.append(sample)


#testing dump all samples to file to verify
import json
with open('training_samples.json', 'w', encoding='utf-8') as f:
    json.dump(samples, f, indent=2, ensure_ascii=False)


CHECK DATA FOR PII  
train and run pii model here

https://github.com/microsoft/presidio - pretrained so don't need to find another dataset

In [ ]:
# install dependecies
# !pip install presidio_analyzer presidio_anonymizer
# !python -m spacy download en_core_web_lg

In [ ]:
from presidio_analyzer import AnalyzerEngine
from collections import defaultdict
from tqdm import tqdm

# initialize analyzer
analyzer = AnalyzerEngine()

# specific countries and years are necessary to the survey data
# do not flag these as PII
excludeWords = set(countries)
excludeWords.update(['2011', '2012', '2013', '2014', '2015', '2016'])

print("Sample of exclude words:", list(excludeWords)[:10])
print("Total words in exclude list:", len(excludeWords))

# initialize storage for pii
potentialPII = []
piiSummary = defaultdict(int)

# iterate through every sample
for idx, sample in enumerate(tqdm(samples, desc="progress")):

    # get input question and target
    inputText = sample["input"]
    targetText = sample["target"]

    # analyze input and target
    inputRes = analyzer.analyze(text=inputText, language='en')
    targetRes = analyzer.analyze(text=targetText, language='en')

    # filter out exclude list from text matches
    inputRes = [r for r in inputRes if not any(inputText[r.start:r.end] in word or word in inputText[r.start:r.end] for word in excludeWords)] 
    targetRes = [r for r in targetRes if targetText[r.start:r.end] not in excludeWords] 

    if inputRes or targetRes:
        print(f"\nDetected in sample {idx}:")
        for r in inputRes:
            print(f"  Input: '{inputText[r.start:r.end]}' (type: {r.entity_type})")
        for r in targetRes:
            print(f"  Target: '{targetText[r.start:r.end]}' (type: {r.entity_type})")
        
        if idx > 5:  # Just check first few samples
            break


    # if pii is found
    if inputRes or targetRes:

        # store results and append main list
        finding = {
            "sampleIDX": idx,
            "input": inputText,
            "target": targetText,
            "inputPII": [{"type": r.entity_type, "text": inputText[r.start:r.end], "score": r.score} for r in inputRes],
            "targetPII": [{"type": r.entity_type, "text": targetText[r.start:r.end], "score": r.score} for r in targetRes]
        }
        potentialPII.append(finding)

# dump to file
import json
with open('potentialPII.json', 'w', encoding='utf-8') as f:
    json.dump(potentialPII, f, indent=2, ensure_ascii=False)


Sample of exclude words: ['Spain', 'Comoros', 'Latvia', 'Lesotho', 'Jersey', 'Romania', 'Canada', 'Australia', 'Singapore', 'Cayman Islands']
Total words in exclude list: 167


progress:   0%|          | 8/107833 [00:00<39:18, 45.72it/s]


Detected in sample 2:
  Input: 'Antigua' (type: LOCATION)
  Input: 'Barbuda' (type: LOCATION)

Detected in sample 8:
  Target: 'the Republic of Azerbaijan' (type: LOCATION)
